In [1]:
import numpy as np
import pandas as pd

import cvxpy as cp

# using pandapower here as data source
import pandapower.networks

In [16]:
# get and set-up the 33bus distribution system model
# assumes radial network 

# some settings
BASE_MVA = 10
BASE_KV = 12.66 

# load network data and compute per-unit values
network_data = pandapower.networks.case33bw()

# per unit transform
v_base = BASE_KV * 1e3
s_base = BASE_MVA * 1e6
def compute_line_pu(row):
    row['r_pu'] = (row['r_ohm_per_km']/(v_base**2/s_base))*row['length_km']
    row['x_pu'] = (row['x_ohm_per_km']/(v_base**2/s_base))*row['length_km']
    return row
network_data.line = network_data.line.apply(compute_line_pu, axis=1)
network_data.line = network_data.line.sort_values('to_bus')
network_data.load[['p_pu', 'q_pu']] = network_data.load[['p_mw', 'q_mvar']].apply(lambda val: (val*1e6)/s_base)

# this data set does not have shunt suceptances, I am including it here for completeness as usually done in this data set
network_data.bus['gs_pu'] = np.zeros(len(network_data.bus))
network_data.bus['bs_pu'] = np.zeros(len(network_data.bus))

# get data as dicts for mor efficient iterations
buses = network_data.bus.to_dict('records')
lines = network_data.line.to_dict('records')
lines = [line for line in lines if line['in_service']] # remove lines that are not in service
substation = network_data.ext_grid.to_dict('records')
loads = network_data.load.to_dict('records')

# add pv and battery systems 
# TODO test feasibility 
# TODO try and find actual data to overlay the feeder with
pv_buses = np.array([19 , 22 , 16 , 17 , 18 , 24 , 25 , 27 , 31 ]) - 1 
pv_s_pu =  np.array([400, 240, 100, 100, 100, 220, 400, 150, 150])*1e3/s_base  # written in kVA   
battery_buses =   np.array([19 , 18, 25 , 24 ]) -1 
battery_bmax_pu = np.array([250, 50, 200, 100])*1e3/s_base # written in kWh
battery_pmax_pu = np.array([50 , 22, 50 , 50 ])*1e3/s_base # written in kW
# all batteries are co-located

pvs = [{'bus': pv_buses[i], 's_max_pu': pv_s_pu[i]} for i in range(len(pv_buses))]
batteries = [{'bus': battery_buses[i], 'b_max_pu': battery_bmax_pu[i], 'p_max_pu': battery_pmax_pu[i]} for i in range(len(battery_buses))]

# map some data to buses
for bus in buses:
    bus.update({'load_at_bus': []})
    bus.update({'pv_at_bus': []})
    bus.update({'battery_at_bus': []})
    bus.update({'children': []})
    bus.update({'ancestor': None})
for load_i, load in enumerate(loads):
    buses[load['bus']]['load_at_bus'].append(load_i)
for pv_i, pv in enumerate(pvs):
    buses[pv['bus']]['pv_at_bus'].append(pv_i)
for bat_i, bat in enumerate(batteries):
    buses[bat['bus']]['battery_at_bus'].append(bat_i)
for line_i, line in enumerate(lines):
    buses[line['from_bus']]['children'].append(line['to_bus'])
    buses[line['to_bus']]['ancestor'] = line['from_bus']

# cost data
cE = 100 # enegy tariff [$/MWh] 
cS = 80  # cost at substation [$/MWh] 
cF = 30  # feed-in tariff [$/MWh]
cQ = 30  # reactive power service reimbursement [$/MVA]
cC = 60  # curtailment fee [$/MW] 


# for now just assume all pvs have the same %age availability
Pav = 1 * pv_s_pu 
for i,pv in enumerate(pvs):
    pv.update({'p_av': Pav[i]})

In [17]:
# network_data = pandapower.networks.case33bw()
network_data.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent,r_pu,x_pu
0,None,None,0,1,1.0,0.0922,0.0470,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.005753,0.002932
1,None,None,1,2,1.0,0.4930,0.2511,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.030760,0.015667
2,None,None,2,3,1.0,0.3660,0.1864,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.022836,0.011630
3,None,None,3,4,1.0,0.3811,0.1941,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.023778,0.012110
4,None,None,4,5,1.0,0.8190,0.7070,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.051099,0.044112
5,None,None,5,6,1.0,0.1872,0.6188,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.011680,0.038608
32,None,None,20,7,1.0,2.0000,2.0000,0.0,0.0,99999.0,1.0,1,ol,False,100.0,0.124785,0.124785
6,None,None,6,7,1.0,0.7114,0.2351,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.044386,0.014668
7,None,None,7,8,1.0,1.0300,0.7400,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.064264,0.046170
8,None,None,8,9,1.0,1.0440,0.7400,0.0,0.0,99999.0,1.0,1,ol,True,100.0,0.065138,0.046170


In [5]:
# build the problem for a single time step for now (1 time step = 1h, so power and energy are the same quantity)
# here in "direct" form, a matrix form may have advantages for some applicatons
N_lines = len(lines)
N_buses = len(buses)
N_pvs = len(pvs)
N_batts = len(batteries)

U0 = 1. # target voltage at substation
Umin = 0.9
Umax = 1.1

# variables
# control variables
alpha = cp.Variable(N_pvs, nonneg=True, name="alpha") # curtailment
qc = cp.Variable(N_pvs) # reactive power contribution
pb = cp.Variable(N_batts, name="pb") # battery power
# implicit states as variables
p_ext = cp.Variable() # active power from external grid
q_ext = cp.Variable() # reactive power from external grid
fp = cp.Variable(N_lines) # active power flow
fq = cp.Variable(N_lines) # reactive power flow
u = cp.Variable(N_buses, name="u") # voltage magnitude squared
l = cp.Variable(N_lines) # current magnitude squared
b = cp.Variable(N_batts) # battery soc
#auxillary for objective
net_consumption = cp.Variable(N_buses, nonneg=True, name="net_consumption")
net_feed_in = cp.Variable(N_buses, nonneg=True, name="net_feed_in")

consts = []
# constraints for power flow model
for bus_i, bus in enumerate(buses):
    if bus_i == 0:        
        # injection from substation
        consts.append(p_ext == sum(fp[i-1] + lines[i-1]['r_pu']*l[i-1] for i in bus['children']) + u[bus_i]*bus['gs_pu']) 
        consts.append(q_ext == sum(fq[i-1] + lines[i-1]['x_pu']*l[i-1] for i in bus['children']) + u[bus_i]*bus['bs_pu'])
        consts.append(net_consumption[bus_i] == 0)
        consts.append(net_feed_in[bus_i] == 0)
    else:     
        net_p = 0
        net_q = 0
        for i in bus['load_at_bus']:
            net_p += -loads[i]['p_pu']
            net_q += -loads[i]['q_pu']
        for i in bus['pv_at_bus']:
            net_p += (1-alpha[i])*pvs[i]['p_av']
            net_q += qc[i]
        for i in bus['battery_at_bus']:
            net_p += -pb[i]
        # flow definition
        consts.append(fp[bus_i-1] + net_p == sum(fp[i-1] + lines[i-1]['r_pu']*l[i-1] for i in bus['children']) + u[bus_i]*bus['gs_pu'])
        consts.append(fq[bus_i-1] + net_q == sum(fq[i-1] + lines[i-1]['x_pu']*l[i-1] for i in bus['children']) + u[bus_i]*bus['bs_pu'])
        # voltage definition
        consts.append(u[bus['ancestor']] == u[bus_i] + 2*(lines[bus_i-1]['r_pu']*fp[bus_i-1] + lines[bus_i-1]['x_pu']*fq[bus_i-1]) + 
                      (lines[bus_i-1]['r_pu']**2 + lines[bus_i-1]['x_pu']**2)*l[bus_i-1])
        consts.append(cp.quad_over_lin(cp.hstack([fp[bus_i-1], fq[bus_i-1]]), u[bus_i]) <= l[bus_i-1])
        consts.append(net_consumption[bus_i] <= -net_p)
        consts.append(net_feed_in[bus_i] >= net_p)
        
# operational constraints
for bus_i, bus in enumerate(buses):
    if bus_i == 0:  
        consts.append(u[bus_i] == 1.03)
        # consts.append(u[bus_i] >= 0.95)
        # consts.append(u[bus_i] <= 1.05)
    else:
        consts.append(u[bus_i] >= Umin)
        consts.append(u[bus_i] <= Umax)
for pv_i, pv in enumerate(pvs):
    consts.append(cp.sum_squares(cp.hstack([(1-alpha[pv_i])*pv['p_av'], qc[pv_i]])) <= pv['s_max_pu']**2)
    consts.append(alpha[pv_i] <= 1)
for bat_i, bat in enumerate(batteries):
    # no energy constraint here because single time step
    consts.append(pb[bat_i] <= bat['p_max_pu'])
    consts.append(pb[bat_i] >= -bat['p_max_pu'])
        

# objective
grid_cost = cS * p_ext 
revenue = cE * cp.sum(net_consumption)
feed_in_payments = cF * cp.sum(net_feed_in)
reactive_power_payments = cQ * cp.norm(qc, 1)
curtailment_payments = cC * sum(alpha[i] * pv['p_av'] for i,pv in enumerate(pvs))
substation_voltage = cp.norm(1-u[0])
objective = grid_cost - revenue + feed_in_payments + reactive_power_payments + curtailment_payments
objective = grid_cost + 100*substation_voltage - revenue + feed_in_payments + reactive_power_payments + curtailment_payments


# problem
model = cp.Problem(cp.Minimize(objective), consts)

In [6]:
# solve it
model.solve(solver='GUROBI', verbose=False)
print(model.status)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-07
optimal


In [7]:
# optimal but nothing interesting happens. 
# prob to little RES / too much battery

In [8]:
model.var_dict['u'].value

array([1.03      , 1.02558995, 1.00457162, 0.99253087, 0.98083255,
       0.95194237, 0.94597029, 0.93881041, 0.92995584, 0.92202374,
       0.92095962, 0.91919128, 0.9119336 , 0.90927476, 0.90829468,
       0.90780605, 0.90669179, 0.90642154, 1.02490718, 1.01948994,
       1.01855215, 1.01808422, 1.0005127 , 0.99336949, 0.99083486,
       0.94906521, 0.94530342, 0.92638983, 0.9130205 , 0.90751885,
       0.90219951, 0.90052007, 0.90000003])

In [5]:
network_data.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable,p_pu,q_pu
0,None,1,0.100,0.060,0.0,0.0,NaN,1.0,True,None,False,0.0100,0.0060
1,None,2,0.090,0.040,0.0,0.0,NaN,1.0,True,None,False,0.0090,0.0040
2,None,3,0.120,0.080,0.0,0.0,NaN,1.0,True,None,False,0.0120,0.0080
3,None,4,0.060,0.030,0.0,0.0,NaN,1.0,True,None,False,0.0060,0.0030
4,None,5,0.060,0.020,0.0,0.0,NaN,1.0,True,None,False,0.0060,0.0020
5,None,6,0.200,0.100,0.0,0.0,NaN,1.0,True,None,False,0.0200,0.0100
6,None,7,0.200,0.100,0.0,0.0,NaN,1.0,True,None,False,0.0200,0.0100
7,None,8,0.060,0.020,0.0,0.0,NaN,1.0,True,None,False,0.0060,0.0020
8,None,9,0.060,0.020,0.0,0.0,NaN,1.0,True,None,False,0.0060,0.0020
9,None,10,0.045,0.030,0.0,0.0,NaN,1.0,True,None,False,0.0045,0.0030
